In [1]:
import pandas as pd
from joblib import load
import numpy as np


lr_model = load("model\\linear_regression.joblib")
ohe = load("model\\onehot_encoder.pkl")

In [6]:
new_data = pd.DataFrame([{
    'carrier': '2S',
    'departure_date': '2024-12-01',
    'origin': 'HN',
    'destination': 'SG'
}])

In [7]:
new_data['departure_date'] = pd.to_datetime(new_data['departure_date'], errors='coerce')
new_data['year'] = new_data['departure_date'].dt.year
new_data['month'] = new_data['departure_date'].dt.month
new_data['day'] = new_data['departure_date'].dt.day
new_data['day_of_week'] = new_data['departure_date'].dt.dayofweek
new_data['quarter'] = new_data['departure_date'].dt.quarter

new_data['sin_month'] = np.sin(2 * np.pi * new_data['month'] / 12)
new_data['cos_month'] = np.cos(2 * np.pi * new_data['month'] / 12)
new_data['sin_dayofweek'] = np.sin(2 * np.pi * new_data['day_of_week'] / 7)
new_data['cos_dayofweek'] = np.cos(2 * np.pi * new_data['day_of_week'] / 7)

def get_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "autumn"

new_data['season'] = new_data['month'].apply(get_season)


categorical_cols = ['season','carrier', 'origin', 'destination', 'year','quarter','month', 'day']

X_ = new_data[['sin_month', 'cos_month','sin_dayofweek', 'cos_dayofweek']]
X_cat = ohe.transform(new_data[categorical_cols])
cat_feature_names = ohe.get_feature_names_out(categorical_cols)
X_cat_df = pd.DataFrame(X_cat, columns=cat_feature_names, index=new_data.index)


X_encoded = pd.concat([X_cat_df,X_], axis=1)
X_encoded.shape


(1, 4853)

In [8]:
y_pred = lr_model.predict(X_encoded)
print(f"✅ Dự đoán economy: {y_pred[0]:.4f} gram CO2")

✅ Dự đoán economy: 71623.1912 gram CO2
